# Mobile Customer Churn

In this Portfolio task you will work with some (fake but realistic) data on Mobile Customer Churn.  Churn is where
a customer leaves the mobile provider.   The goal is to build a simple predictive model to predict churn from available features. 

The data was generated (by Hume Winzar at Macquarie) based on a real dataset provided by Optus.  The data is simulated but the column headings are the same. (Note that I'm not sure if all of the real relationships in this data are preserved so you need to be cautious in interpreting the results of your analysis here).  

The data is provided in file `MobileCustomerChurn.csv` and column headings are defined in a file `MobileChurnDataDictionary.csv` (store these in the `files` folder in your project).

Your high level goal in this notebook is to try to build and evaluate a __predictive model for churn__ - predict the value of the CHURN_IND field in the data from some of the other fields.  Note that the three `RECON` fields should not be used as they indicate whether the customer reconnected after having churned. 

__Note:__ you are not being evaluated on the _accuracy_ of the model but on the _process_ that you use to generate it.  You can use a simple model such as Logistic Regression for this task or try one of the more advanced methods covered in recent weeks.  Explore the data, build a model using a selection of features and then do some work on finding out which features provide the most accurate results.  

In [118]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [119]:
churn = pd.read_csv("MobileCustomerChurn.csv", na_values=["NA", "#VALUE!"], index_col='INDEX')
churn.head()

,CUST_ID,ACCOUNT_TENURE,ACCT_CNT_SERVICES,AGE,CFU,SERVICE_TENURE,PLAN_ACCESS_FEE,BYO_PLAN_STATUS,PLAN_TENURE,MONTHS_OF_CONTRACT_REMAINING,...,CONTRACT_STATUS,PREV_CONTRACT_DURATION,HANDSET_USED_BRAND,CHURN_IND,MONTHLY_SPEND,COUNTRY_METRO_REGION,STATE,RECON_SMS_NEXT_MTH,RECON_TELE_NEXT_MTH,RECON_EMAIL_NEXT_MTH
INDEX,,,,,,,,,,,,,,,,,,,,,
1,1,46,1,30.0,CONSUMER,46,54.54,NON BYO,15,0,...,OFF-CONTRACT,24,SAMSUNG,1,61.40,COUNTRY,WA,NaN,NaN,NaN
2,2,60,3,55.0,CONSUMER,59,54.54,NON BYO,5,0,...,OFF-CONTRACT,24,APPLE,1,54.54,METRO,NSW,NaN,NaN,NaN
3,5,65,1,29.0,CONSUMER,65,40.90,BYO,15,0,...,OFF-CONTRACT,12,APPLE,1,2.50,COUNTRY,WA,NaN,NaN,NaN
4,6,31,1,51.0,CONSUMER,31,31.81,NON BYO,31,0,...,OFF-CONTRACT,24,APPLE,1,6.48,COUNTRY,VIC,NaN,NaN,NaN
5,8,95,1,31.0,CONSUMER,95,54.54,NON BYO,0,0,...,OFF-CONTRACT,24,APPLE,1,100.22,METRO,NSW,NaN,NaN,NaN


In [120]:
#Using Logistical Regression
#Explore the data, build a model using a selection of features and then do some work on finding out which features provide the most accurate results.

churn = churn.dropna(subset=["STATE","COUNTRY_METRO_REGION","AGE"]) #DROP NA VALUES in these columns
churn.isna().sum() #check if dropped (Dont drop RECON since it will be dropped anyways when testing/training)

CUST_ID                             0
ACCOUNT_TENURE                      0
ACCT_CNT_SERVICES                   0
AGE                                 0
CFU                                 0
SERVICE_TENURE                      0
PLAN_ACCESS_FEE                     0
BYO_PLAN_STATUS                     0
PLAN_TENURE                         0
MONTHS_OF_CONTRACT_REMAINING        0
LAST_FX_CONTRACT_DURATION           0
CONTRACT_STATUS                     0
PREV_CONTRACT_DURATION              0
HANDSET_USED_BRAND                  0
CHURN_IND                           0
MONTHLY_SPEND                       0
COUNTRY_METRO_REGION                0
STATE                               0
RECON_SMS_NEXT_MTH              17763
RECON_TELE_NEXT_MTH             17763
RECON_EMAIL_NEXT_MTH            17763
dtype: int64

In [162]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn import metrics
from sklearn.feature_selection import RFE

train, test = train_test_split(churn, test_size=0.2, random_state=42) #more training, less testing
print(train.shape)   #fixing random_state gives same result
print(test.shape)

(36903, 21)
(9226, 21)


In [163]:
set(churn['CHURN_IND']) #Checking CHURN_IND classes

{0, 1}

In [195]:
# Logistic Regression model function (testing and evaluation)

def RegressionModel(x_train,y_train,x_test,y_test):
    model = LogisticRegression(solver='lbfgs', max_iter=1000)
    model.fit(x_train,y_train)
    
    #Testing
    y_pred = model.predict(x_test)
    print(y_pred)
    
    print("\nActual CHURN_IND sample values from training dataset:")
    print(y_test[:5])

    print("\nCorresponding Predicted CHURN_IND samples:")
    print(y_pred[:5])
    
    #Evaluation of model
    yhat = model.predict(x_test) 
    testing_accuracy = metrics.accuracy_score(y_test, yhat)
    print("\n\nTesting accuracy: {:.2f}%".format(testing_accuracy))

    train_preds = model.predict(x_train)
    training_accuracy = metrics.accuracy_score(y_train, train_preds)
    print("Training accuracy: {:.2f}%".format(training_accuracy))
    
    
    lr_model = LogisticRegression()
    rfe = RFE(estimator=lr_model, n_features_to_select=5, step=1)
    rfe.fit(x_train,y_train)

    print("\nConfusion matrix on test set: ")
    print(confusion_matrix(y_test, yhat)) #1 = true positive

    #TP = 1, FP = 2, FN = 3, TN = 4 (confusion matrix)
    #[TP FP 
    # FN TN]

    #compute accuracy score on confusion matrix
    y_test_hat = rfe.predict(x_test) #predicting, so use x_test
    print("Accuracy of confusion matrix: {:.2f}%".format(accuracy_score(y_test, y_test_hat)))

    #ROC-AUC SCORE
    score = metrics.roc_auc_score(y_test,y_test_hat)
    print("ROC score: {:.2f}%".format(score))

In [282]:
#Define some parameters

x_train1 = train.drop(['CUST_ID','CHURN_IND','BYO_PLAN_STATUS','CFU','CONTRACT_STATUS','HANDSET_USED_BRAND',
                       'COUNTRY_METRO_REGION','STATE','RECON_SMS_NEXT_MTH','RECON_TELE_NEXT_MTH',
                       'RECON_EMAIL_NEXT_MTH'],axis=1)
x_test1 = test.drop(['CUST_ID','CHURN_IND','BYO_PLAN_STATUS','CFU','CONTRACT_STATUS','HANDSET_USED_BRAND',
                     'COUNTRY_METRO_REGION','STATE','RECON_SMS_NEXT_MTH','RECON_TELE_NEXT_MTH',
                     'RECON_EMAIL_NEXT_MTH'],axis=1)#All numerical fields

x_train2 = train.drop(['ACCT_CNT_SERVICES','ACCOUNT_TENURE','SERVICE_TENURE','PLAN_ACCESS_FEE','PLAN_TENURE',
                       'MONTHS_OF_CONTRACT_REMAINING','LAST_FX_CONTRACT_DURATION','PREV_CONTRACT_DURATION', 
                       'CUST_ID','CHURN_IND','BYO_PLAN_STATUS','CFU','CONTRACT_STATUS','HANDSET_USED_BRAND',
                       'COUNTRY_METRO_REGION','STATE','RECON_SMS_NEXT_MTH','RECON_TELE_NEXT_MTH',
                       'RECON_EMAIL_NEXT_MTH'],axis=1)
x_test2 = test.drop(['ACCT_CNT_SERVICES','ACCOUNT_TENURE','SERVICE_TENURE','PLAN_ACCESS_FEE','PLAN_TENURE',
                       'MONTHS_OF_CONTRACT_REMAINING','LAST_FX_CONTRACT_DURATION','PREV_CONTRACT_DURATION', 
                       'CUST_ID','CHURN_IND','BYO_PLAN_STATUS','CFU','CONTRACT_STATUS','HANDSET_USED_BRAND',
                       'COUNTRY_METRO_REGION','STATE','RECON_SMS_NEXT_MTH','RECON_TELE_NEXT_MTH',
                       'RECON_EMAIL_NEXT_MTH'],axis=1) #age and monthly spend

x_train3 = train.drop(['MONTHLY_SPEND','AGE','SERVICE_TENURE','PLAN_ACCESS_FEE','PLAN_TENURE',
                       'MONTHS_OF_CONTRACT_REMAINING','LAST_FX_CONTRACT_DURATION','PREV_CONTRACT_DURATION', 
                       'CUST_ID','CHURN_IND','BYO_PLAN_STATUS','CFU','CONTRACT_STATUS','HANDSET_USED_BRAND',
                       'COUNTRY_METRO_REGION','STATE','RECON_SMS_NEXT_MTH','RECON_TELE_NEXT_MTH',
                       'RECON_EMAIL_NEXT_MTH'],axis=1)
x_test3 = test.drop(['MONTHLY_SPEND','AGE','SERVICE_TENURE','PLAN_ACCESS_FEE','PLAN_TENURE',
                       'MONTHS_OF_CONTRACT_REMAINING','LAST_FX_CONTRACT_DURATION','PREV_CONTRACT_DURATION', 
                       'CUST_ID','CHURN_IND','BYO_PLAN_STATUS','CFU','CONTRACT_STATUS','HANDSET_USED_BRAND',
                       'COUNTRY_METRO_REGION','STATE','RECON_SMS_NEXT_MTH','RECON_TELE_NEXT_MTH',
                       'RECON_EMAIL_NEXT_MTH'],axis=1) #Account information

x_train4 = train.drop(['MONTHLY_SPEND','AGE','SERVICE_TENURE','PLAN_ACCESS_FEE','PLAN_TENURE',
                       'ACCOUNT_TENURE','ACCT_CNT_SERVICES', 
                       'CUST_ID','CHURN_IND','BYO_PLAN_STATUS','CFU','CONTRACT_STATUS','HANDSET_USED_BRAND',
                       'COUNTRY_METRO_REGION','STATE','RECON_SMS_NEXT_MTH','RECON_TELE_NEXT_MTH',
                       'RECON_EMAIL_NEXT_MTH'],axis=1)
x_test4 = test.drop(['MONTHLY_SPEND','AGE','SERVICE_TENURE','PLAN_ACCESS_FEE','PLAN_TENURE',
                       'ACCOUNT_TENURE','ACCT_CNT_SERVICES', 
                       'CUST_ID','CHURN_IND','BYO_PLAN_STATUS','CFU','CONTRACT_STATUS','HANDSET_USED_BRAND',
                       'COUNTRY_METRO_REGION','STATE','RECON_SMS_NEXT_MTH','RECON_TELE_NEXT_MTH',
                       'RECON_EMAIL_NEXT_MTH'],axis=1) #Monthly contract info (excluding monthly spending)

x_train5 = train.drop(['AGE','ACCOUNT_TENURE','ACCT_CNT_SERVICES',
                       'MONTHS_OF_CONTRACT_REMAINING','LAST_FX_CONTRACT_DURATION','PREV_CONTRACT_DURATION', 
                       'CUST_ID','CHURN_IND','BYO_PLAN_STATUS','CFU','CONTRACT_STATUS','HANDSET_USED_BRAND',
                       'COUNTRY_METRO_REGION','STATE','RECON_SMS_NEXT_MTH','RECON_TELE_NEXT_MTH',
                       'RECON_EMAIL_NEXT_MTH'],axis=1)
x_test5 = test.drop(['AGE','ACCOUNT_TENURE','ACCT_CNT_SERVICES',
                       'MONTHS_OF_CONTRACT_REMAINING','LAST_FX_CONTRACT_DURATION','PREV_CONTRACT_DURATION', 
                       'CUST_ID','CHURN_IND','BYO_PLAN_STATUS','CFU','CONTRACT_STATUS','HANDSET_USED_BRAND',
                       'COUNTRY_METRO_REGION','STATE','RECON_SMS_NEXT_MTH','RECON_TELE_NEXT_MTH',
                       'RECON_EMAIL_NEXT_MTH'],axis=1) #Plans/service information and monthly spending

x_train6 = train.drop(['AGE','ACCOUNT_TENURE','ACCT_CNT_SERVICES',
                       'CUST_ID','CHURN_IND','BYO_PLAN_STATUS','CFU','CONTRACT_STATUS','HANDSET_USED_BRAND',
                       'COUNTRY_METRO_REGION','STATE','RECON_SMS_NEXT_MTH','RECON_TELE_NEXT_MTH',
                       'RECON_EMAIL_NEXT_MTH'],axis=1)
x_test6 = test.drop(['AGE','ACCOUNT_TENURE','ACCT_CNT_SERVICES',
                       'CUST_ID','CHURN_IND','BYO_PLAN_STATUS','CFU','CONTRACT_STATUS','HANDSET_USED_BRAND',
                       'COUNTRY_METRO_REGION','STATE','RECON_SMS_NEXT_MTH','RECON_TELE_NEXT_MTH',
                       'RECON_EMAIL_NEXT_MTH'],axis=1) #Plans/service information and monthly contract info   

x_train7 = train.drop(['ACCT_CNT_SERVICES','AGE','MONTHLY_SPEND','PLAN_ACCESS_FEE','MONTHLY_SPEND',
                       'MONTHS_OF_CONTRACT_REMAINING','LAST_FX_CONTRACT_DURATION','PREV_CONTRACT_DURATION', 
                       'CUST_ID','CHURN_IND','BYO_PLAN_STATUS','CFU','CONTRACT_STATUS','HANDSET_USED_BRAND',
                       'COUNTRY_METRO_REGION','STATE','RECON_SMS_NEXT_MTH','RECON_TELE_NEXT_MTH',
                       'RECON_EMAIL_NEXT_MTH'],axis=1)
x_test7 = test.drop(['ACCT_CNT_SERVICES','AGE','MONTHLY_SPEND','PLAN_ACCESS_FEE','MONTHLY_SPEND',
                       'MONTHS_OF_CONTRACT_REMAINING','LAST_FX_CONTRACT_DURATION','PREV_CONTRACT_DURATION', 
                       'CUST_ID','CHURN_IND','BYO_PLAN_STATUS','CFU','CONTRACT_STATUS','HANDSET_USED_BRAND',
                       'COUNTRY_METRO_REGION','STATE','RECON_SMS_NEXT_MTH','RECON_TELE_NEXT_MTH',
                       'RECON_EMAIL_NEXT_MTH'],axis=1) #Tenure information

#y_train/test will always be the same (CHURN_IND)
y_train1 = train['CHURN_IND']
y_test1 = test['CHURN_IND']

In [283]:
print("Listed below are different models predicting CHURN_IND with different fields to test/train with."
      "\nIn ascending order from least accurate to most accurate")

Listed below are different models predicting CHURN_IND with different fields to test/train with.
In ascending order from least accurate to most accurate


In [284]:
print("7.Model predicting CHURN_IND from Account information:")
RegressionModel(x_train3,y_train1,x_test3,y_test1)

7.Model predicting CHURN_IND from Account information:
[0 0 0 ... 0 0 0]

Actual CHURN_IND sample values from training dataset:
INDEX
20090    0
29283    0
23036    1
14911    0
33262    0
Name: CHURN_IND, dtype: int64

Corresponding Predicted CHURN_IND samples:
[0 0 0 0 0]


Testing accuracy: 0.62%
Training accuracy: 0.61%

Confusion matrix on test set: 
[[5677    0]
 [3549    0]]
Accuracy of confusion matrix: 0.62%
ROC score: 0.50%


In [285]:
print("6.Model predicting CHURN_IND from age and monthly spending:")
RegressionModel(x_train2,y_train1,x_test2,y_test1)

6.Model predicting CHURN_IND from age and monthly spending:
[0 0 0 ... 0 0 0]

Actual CHURN_IND sample values from training dataset:
INDEX
20090    0
29283    0
23036    1
14911    0
33262    0
Name: CHURN_IND, dtype: int64

Corresponding Predicted CHURN_IND samples:
[0 0 0 0 0]


Testing accuracy: 0.66%
Training accuracy: 0.66%

Confusion matrix on test set: 
[[5477  200]
 [2950  599]]
Accuracy of confusion matrix: 0.66%
ROC score: 0.57%


In [286]:
print("5.Model predicting CHURN_IND from Monthly contract info (excluding monthly spending):")
RegressionModel(x_train4,y_train1,x_test4,y_test1)

5.Model predicting CHURN_IND from Monthly contract info (excluding monthly spending):
[0 0 1 ... 1 1 0]

Actual CHURN_IND sample values from training dataset:
INDEX
20090    0
29283    0
23036    1
14911    0
33262    0
Name: CHURN_IND, dtype: int64

Corresponding Predicted CHURN_IND samples:
[0 0 1 0 0]


Testing accuracy: 0.64%
Training accuracy: 0.65%

Confusion matrix on test set: 
[[4545 1132]
 [2170 1379]]
Accuracy of confusion matrix: 0.64%
ROC score: 0.59%


In [287]:
print("4.Model predicting CHURN_IND from Plans/service information and monthly contract info:")
RegressionModel(x_train5,y_train1,x_test5,y_test1)

4.Model predicting CHURN_IND from Plans/service information and monthly contract info:
[0 0 0 ... 1 0 0]

Actual CHURN_IND sample values from training dataset:
INDEX
20090    0
29283    0
23036    1
14911    0
33262    0
Name: CHURN_IND, dtype: int64

Corresponding Predicted CHURN_IND samples:
[0 0 0 0 0]


Testing accuracy: 0.68%
Training accuracy: 0.68%

Confusion matrix on test set: 
[[5092  585]
 [2396 1153]]
Accuracy of confusion matrix: 0.68%
ROC score: 0.61%


In [288]:
print("3.Model predicting CHURN_IND from Tenure information:")
RegressionModel(x_train7,y_train1,x_test7,y_test1)

3.Model predicting CHURN_IND from Tenure information:
[0 0 0 ... 1 0 0]

Actual CHURN_IND sample values from training dataset:
INDEX
20090    0
29283    0
23036    1
14911    0
33262    0
Name: CHURN_IND, dtype: int64

Corresponding Predicted CHURN_IND samples:
[0 0 0 0 0]


Testing accuracy: 0.63%
Training accuracy: 0.63%

Confusion matrix on test set: 
[[5074  603]
 [2786  763]]
Accuracy of confusion matrix: 0.63%
ROC score: 0.55%


In [289]:
print("2.Model predicting CHURN_IND from Plans/service/Monthly info:")
RegressionModel(x_train6,y_train1,x_test6,y_test1)

2.Model predicting CHURN_IND from Plans/service/Monthly info:
[0 0 1 ... 1 0 0]

Actual CHURN_IND sample values from training dataset:
INDEX
20090    0
29283    0
23036    1
14911    0
33262    0
Name: CHURN_IND, dtype: int64

Corresponding Predicted CHURN_IND samples:
[0 0 1 0 0]


Testing accuracy: 0.70%
Training accuracy: 0.71%

Confusion matrix on test set: 
[[4624 1053]
 [1703 1846]]
Accuracy of confusion matrix: 0.70%
ROC score: 0.66%


In [290]:
print("1.Model predicting CHURN_IND from all numerical fields:")
RegressionModel(x_train1,y_train1,x_test1,y_test1)

1.Model predicting CHURN_IND from all numerical fields:
[0 0 1 ... 1 0 0]

Actual CHURN_IND sample values from training dataset:
INDEX
20090    0
29283    0
23036    1
14911    0
33262    0
Name: CHURN_IND, dtype: int64

Corresponding Predicted CHURN_IND samples:
[0 0 1 0 0]


Testing accuracy: 0.70%
Training accuracy: 0.71%


/Users/joeb/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/joeb/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logisti


Confusion matrix on test set: 
[[4606 1071]
 [1663 1886]]
Accuracy of confusion matrix: 0.70%
ROC score: 0.67%


In [291]:
print("The above models clearly show that the model using all numerical fields to predict CHURN_IND field is the "
     "most\naccurate. Models using fields such as Tenure or Plans/service/Monthly info are usually on average "
     "more accurate\nthan models using fields such as only account information, age and monthly spending.")

The above models clearly show that the model using all numerical fields to predict CHURN_IND field is the most
accurate. Models using fields such as Tenure or Plans/service/Monthly info are usually on average more accurate
than models using fields such as only account information, age and monthly spending.
